<a href="https://colab.research.google.com/github/ychoi-kr/llm-api-prog/blob/main/7_langchain/rag/vector_search_upstage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain_google_genai
!pip install langchain_community
!pip install chromadb
!pip install langchain_upstage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7
ERROR: Ope

In [ ]:
from langchain_upstage import UpstageEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma

In [ ]:
# @title CSV 파일 로드
from google.colab import drive, userdata
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/llm-api-prog/7_langchain/rag")

Mounted at /content/drive


In [ ]:
loader = CSVLoader(
    file_path="./data/한국산업은행_금융 관련 용어_20151231.csv",
    encoding='cp949'
)
pages = loader.load()

print(pages[:2])


[Document(metadata={'source': './data/한국산업은행_금융 관련 용어_20151231.csv', 'row': 0}, page_content='구분: 리스크\n분류: 리스크 개요\n용어: 리스크(Risk)\n설명: 미래수익 또는 자산가치 변동의 불확실성(Uncertainty)으로 인하여 보유자산에서 손실이 발생할 가능성(신용  시장  금리  유동성리스크 등) 또한 부적절하거나 잘못된 내부절차  시스템 오류  직원의 실수·고의 또는 자연재해 등의 사건에 의해 손실이 발생할 가능성 (운영리스크 등)'), Document(metadata={'source': './data/한국산업은행_금융 관련 용어_20151231.csv', 'row': 1}, page_content='구분: 리스크\n분류: 리스크 개요\n용어: 불확실성\n설명: 설사 손실이 발생한다 해도 발생될 것이 확실하고 크기(금액)도 확실히 알 수 있어서 회피 또는 수용하기로 의사결정하고 나면 그것은 더 이상 리스크가 아님')]


In [ ]:
from google.colab import userdata

In [ ]:
# @title 임베딩 모델 로딩
us_model =  UpstageEmbeddings(
    api_key=userdata.get('UPSTAGE_API_KEY'),
    model="solar-embedding-1-large"
)

In [ ]:
# @title 벡터 DB 생성
Chroma.from_documents(pages, us_model, persist_directory="./database")

In [ ]:
# @title 벡터 스토어 객체 로딩
vector_store = Chroma(persist_directory="./database", embedding_function=us_model)

<ipython-input-9-e11dd04ef69d>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(persist_directory="./database", embedding_function=us_model)


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# @title MMR 검색기 생성
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 5, 'fetch_k': 10})

In [ ]:
# @title 프롬프트 템플릿 생성
template = """
[context]: {context}
```
[질의]: {query}
```
[예시]
신용 환산율입니다.
```
위의 [context] 정보 내에서 [질의]에 대해 답변 [예시]와 같이 술어를 붙여서 답하세요.
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# @title 제미나이 객체 생성
from google.colab import userdata
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0, api_key=userdata.get('GOOGLE_API_KEY'))

In [ ]:
# @title 검색 결과 가공하기
def merge_pages(pages):
    merged = "\n\n".join(page.page_content for page in pages)
    print(f"참조 문서 시작==>[\n{merged}\n]<==참조 문서 끝")
    return merged


In [ ]:
# @title 체이닝 구성
chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
# @title 질의응답 테스트
answer = chain.invoke("짧은 기간 동안의 차익을 위해 금융사가 보유하는 걸 뜻하는 용어가 뭐야?")
print(f"answer1: {answer}\n\n")

answer = chain.invoke("트레이딩 포지션이 뭐야?")
print(f"answer2: {answer}")


참조 문서 시작==>[
구분: 리스크
분류: 시장리스크
용어: 트레이딩포지션
설명: 단기 매매차익 획득 등을 목적으로 금융기관이 보유하는 포지션으로 상품채권  상품주식 및 파생상품 등을 말함

구분: 퇴직연금
분류: 
용어: 단기금융시장
설명: 단기(통상 1년 미만) 금융자산을 거래하는 시장

구분: 리스크
분류: 리스크 개요
용어: 유동성리스크
설명: 자금운용과 조달기간의 불일치 또는 예기치 못한 자금유출 등으로 지급불능 상태에 직면하거나  자금의 과부족을 해소하기 위하여 비정상적 고금리 조달 또는 보유자산의 불리한 매각 등으로 손실이 발생할 가능성 (예금을 지급하기 위해 보유자산을 급하게 처분하는 과정에서 발생하는 손실 가능성 등)

구분: 퇴직연금
분류: 
용어: 수익증권
설명: 투자신탁의 수익권을 증권으로 표창한 것

구분: 리스크
분류: 리스크 개요
용어: 리스크(Risk) 관리
설명: 은행은 적절한 수준의 리스크를 부담함으로써 이익을 획득하는 기관으로  은행의 경영 건전성을 손상시키지 않도록 영업상 직면하게 되는 리스크를 적정수준으로 유지·통제하는 활동을 말함
]<==참조 문서 끝
answer1: 트레이딩포지션입니다.



참조 문서 시작==>[
구분: 리스크
분류: 리스크 개요
용어: 포지션
설명: 자금의 과부족 상태  수익이나 자산 등의 규모의 차이를 의미 (예 : 외화자산 - 외화부채 = 환포지션)

구분: 리스크
분류: 시장리스크
용어: 트레이딩포지션
설명: 단기 매매차익 획득 등을 목적으로 금융기관이 보유하는 포지션으로 상품채권  상품주식 및 파생상품 등을 말함

구분: 리스크
분류: 리스크 개요
용어: 시장리스크
설명: 시장지표(금리  주가  환율 등)가 불리한 방향으로 변동할 경우 트레이딩 포지션에서 손실이 발생할 가능성 (은행이 보유한 주식이나 채권의 가격이 하락하여 손실이 발생할 가능성)

구분: 리스크
분류: 시장리스크
용어: Front Office
설명: 트레이더(또는 딜러)가 고객 주문을 받아 대행 거래하거나 직접 자기 계산으로